## Lecture 15 of CMU16-745 (2025)

#### py_wahba.ipynb

In [72]:
import numpy as np
from numpy.linalg import solve,norm
from pydrake import forwarddiff

<hr>

The function definitions in the cell below are the same as in lecture 14, 
with some minor adjustments in L(q), R(q) to fix forwarddiff errors.

In [54]:
def hat(v):
    return np.array([[0,-v[2],v[1]],
                     [v[2],0,-v[0]],
                     [-v[1],v[0],0]])

def L(q):
    global s,v,L_u,L_l
    s = q[0]
    v = q[1:]
    L_u = np.hstack((np.expand_dims(s,axis=0),-v.T))
    L_l = np.hstack((v.reshape(3,1),s*np.eye(3) + hat(v.flatten()))) # add v.reshape()
    return np.vstack((L_u,L_l))

def R(q):
    global R_u, R_l,s,v
    s = q[0]
    v = q[1:]
    R_u = np.hstack((np.expand_dims(s,axis=0),-v.T))
    R_l = np.hstack((v.reshape(3,1),s*np.eye(3) - hat(v.flatten()))) # add v.reshape()
    return np.vstack((R_u,R_l))

T = np.diagflat([1,-1,-1,-1])
H = np.vstack((np.zeros((1,3)),np.eye(3)))

def G(q):
    return L(q)@H

def Q(q):
    return H.T@(R(q).T@L(q))@H


In [26]:
def vec(Mat):
    global M_global
    M_global = Mat
    m, n = np.shape(Mat)[0], np.shape(Mat)[1]
    return np.reshape(Mat,(m*n,1), order='F')

<hr>

In [65]:
# generate a random quaternion
qtrue = np.random.randn(4,1)
qtrue = qtrue/norm(qtrue)
Qtrue = Q(qtrue) # generate equilavent rotation matrix

In [66]:
# generate data
vN = np.random.randn(3,10) # generate some random world-frame vectors

# normalize
for k in range(10):
    vN[:,k] = vN[:,k]/norm(vN[:,k])

vB = Qtrue.T@vN # generate body-frame vectors

In [24]:
def residual(q):
    r = vN - Q(q)@vB
    return vec(r)

In [67]:
# random initial guess
q = np.random.randn(4,1)
q = q/norm(q)

In [68]:
# Gauss-Newton method
phi = np.eye(3)
iter = 0
while np.max(abs(phi)) > 1e-8:
    r = residual(q)
    dr = np.squeeze(forwarddiff.jacobian(residual,q.flatten()))
    gradr = dr@G(q)
    phi = solve(-gradr.T@gradr,gradr.T@r) # 3-parameter update computed w gauss-newton
    q = L(q)@np.vstack((np.sqrt(1-phi.T@phi),phi)) # multiplicative update applied to q
    iter +=1
    

In [69]:
iter

7

In [70]:
q - qtrue

array([[ 0.73675295],
       [-0.72089464],
       [-1.12480143],
       [ 1.29318512]])

In [71]:
q+qtrue

array([[ 0.00000000e+00],
       [-5.55111512e-17],
       [-1.11022302e-16],
       [ 0.00000000e+00]])